![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)

<a href="https://hub.callysto.ca/jupyter/hub/user-redirect/git-pull?repo=https%3A%2F%2Fgithub.com%2Fcallysto%2Fcurriculum-notebooks&branch=master&subPath=SocialStudies/HansardAnalysis/hansard-analysis.ipynb&depth=1" target="_parent"><img src="https://raw.githubusercontent.com/callysto/curriculum-notebooks/master/open-in-callysto-button.svg?sanitize=true" width="123" height="24" alt="Open in Callysto"/></a>

# Callysto's Weekly Data Visualization


## Sunset and Sunrise

### Recommended Grade levels: 5-9

### Instructions

Click "Cell" and select "Run All".

This will import the data and run all the code, so you can see this week's data visualization. Scroll back to the top after you’ve run the cells.

![instructions](https://github.com/callysto/data-viz-of-the-week/blob/main/images/instructions.png?raw=true)

**You don't need to do any coding to view the visualizations**.

The plots generated in this notebook are interactive. You can hover over and click on elements to see more information. 

Email contact@callysto.ca if you experience issues.

### About this Notebook

Callysto's Weekly Data Visualization is a learning resource that aims to develop data literacy skills. We provide Grades 5-12 teachers and students with a data visualization, like a graph, to interpret. This companion resource walks learners through how the data visualization is created and interpreted by a data scientist. 

The steps of the data analysis process are listed below and applied to each weekly topic.

1. Question - What are we trying to answer?
2. Gather - Find the data source(s) you will need. 
3. Organize - Arrange the data, so that you can easily explore it. 
4. Explore - Examine the data to look for evidence to answer the question. This includes creating visualizations. 
5. Interpret - Describe what's happening in the data visualization. 
6. Communicate - Explain how the evidence answers the question. 

## Question

What are the sunset, sunrise, and day lengths of particular areas around the world?

### Goal

Our goal is to show the differences between sunrise, sunset, and day lengths around the world and how timezones can drastically these factors.

### Background

Welcome to the wonderful world of Sunrises and Sunsets! In this vibrant notebook, we embark on an breathtaking journey through the captivating beauty of nature's daily masterpieces, sunrises and sunsets. Have you ever wondered what the sunset times are in Calgary compared to Berlin? We will be exploring these differences throughout this notebook.

## Gather


Sunrise and sunset data was collected through the [Sunset and Sunrise](https://sunrise-sunset.org/api) API, which provides sunset and sunrise times when given a specific latitude and longitude. 

### Code: 

Run the code cells below to import the libraries we need for this project. Libraries are pre-made code that make it easier to analyze our data.

In [ ]:
%pip install -r requirements.txt
import pyodide_http
pyodide_http.patch_all()
import requests
from datetime import *
import pandas as pd
import plotly.express as px
import json
from plotly.subplots import make_subplots
import plotly.graph_objects as go
print('Libraries imported')

[Pandas](https://pandas.pydata.org/) is a library that helps us with data analysis, and [plotly express](https://plotly.com/python/plotly-express/) is a library that helps us to make visualizations. [Requests](https://requests.readthedocs.io/en/latest/) and [json](https://docs.python.org/3/library/json.html) help read the sunset and sunrise data from an external API. If you are not familiar with what an API is or does, it essentially sends information to one party from another. For example, many Twitter bots make use of the Twitter API where twitter sends information these bots may need in order to properly function. [Datetime](https://docs.python.org/3/library/datetime.html) supplies information in regards to manipulating dates and times and [tzfpy](https://pypi.org/project/tzfpy/) allows us to find timezone names by supplying longitude and latitude values. 

Without importing these libraries we would have to use much more code to analyze our data and generate visualizations. We import the libraries with abbreviations, or aliases, so that we have less typing to do in each line of our code below.

### Data
We are using data from the [Sunset and Sunrise](https://sunrise-sunset.org/api) API. Run the code below to populate the data into a dataframe.

#### Import the Data

Let's try inputting the _latitude_ and _longitude_ of Calgary and see what the resulting data is.

In [ ]:
lat= 51.049999
lng= -114.06666

query = {'lat': lat, 'lng': lng, 'date': 'today'}
response = requests.get(url = 'https://api.sunrise-sunset.org/json?', params=query)

data = response.json()
print(f"Information about Calgary: {data}")
df = pd.json_normalize(data['results'])
display(df)

### Comment on the data

The dataframe above is a file structure that allows Python to display data in an easily readable format, similar to a spreadsheet. 

Looking at the data, we have multiple things to take note of. In particular, we see thaty we have access to the `sunrise` and `sunset` for a location at a particular _latitude_ and _longitude_. We also have access to `day_length` which highlights the length of a particular date. However, one particular issue with this dataset is that times are represented in the **UTC timezone**. Let's keep mind of this and _transform_ our data later.

In [ ]:
sunrise = data['results']['sunrise']
sunset = data['results']['sunset']
print(f"Calgary's latitude of {lat} and longitude {lng}, has a corresponding sunrise and sunset times are {sunrise} and {sunset} in the UTC timezone.")

Imagine you're going on a trip around the world, and you want to know how the length of day and night changes depending on where you are. Well, it all has to do with two special lines called latitude and longitude.

Latitude is like a horizontal belt that runs around the Earth. The Equator is the most important line, dividing the Earth into the Northern and Southern Hemispheres.

Longitude is like a set of vertical lines that go from the North Pole to the South Pole. The Prime Meridian, which goes through Greenwich, London, is the main one.

When you're close to the middle of the Earth (the equator), daytime and nighttime are about the same length. But as you move away from the equator towards the North Pole or the South Pole, the days become longer and the nights become shorter. So, the further you are from the equator, the bigger the differences between daytime and nighttime.

## Organize
An important part of the data science process is cleaning up and organizing your data, so it can be useful for finding observations. Part of cleaning involves 
- identifying missing data
- removing missing data
- ensuring the data is all in the same format
- identifying and dealing with outliers. 

In our particular case, we have two main things to keep in mind. One is that our data is currently in the **UTC timezone**. Timezones are super important when it comes to sunset and sunrise times! Imagine you're on an adventure with your friends in different parts of the world. Each place has its own special time. Timezones help us keep track of these different times and make sure everyone knows when the sun will rise and set in their area. You see, the Earth is a big globe, and as it spins around, different parts face the sun at different times. So, when it's morning in one place, it might still be night in another place far away. Timezones help us organize and understand these differences. 

The other issue we need to fix is converting our `sunrise` and `sunset` times from the _12-hour_ format into the _24-hour_ format. This is useful in a number of ways, but we'll go more into depth when we solve the issue directly. 

With a quick search we find out that Calgary is currently in the MDT/MST timezone which is (GMT-6). If you're having a hard time finding out your own local timezone, use [this](https://www.timeanddate.com/worldclock/converter.html) site which helps find the differences between your timezone and UTC.


Using the site listed, we know that MDT/MST is 6 hours behind the **UTC timezone** meaning I have to **subtract** 6 hours from our obtained `sunrise` and `sunset` times in order for times to be in our local timezone. Therefore, I put **-6** in the `hours_to_switch` variable in the cell below.

In [ ]:
# Difference in hours between UTC and MST - Change this value later on in the notebook to the timezone you'd like to swap to
offset_hours = -6

If your timezone was ahead of UTC, instead of subtracting a certain amount of hours, you would have to **add** a certain amount of hours. For example, if my timezone was 4 hours ahead of UTC, I would input **4** into the `hours_to_switch` variable.

In [ ]:
data = {'location': ['Tokyo, Japan', 'London, United Kingdom', 'New York City, United States of America', 
                     'Dubai, United Arab Emirates', 'Hong Kong, China', 'Mumbai, India', 'Bangkok, Thailand',
                     'Paris, France', 'Istanbul, Turkey', 'Seoul, South Korea'],
        'latitude': [35.672855, 51.509865, 40.730610, 25.276987, 22.302711, 19.076090, 13.736717, 48.864716, 41.015137, 37.532600], 
        'longitude': [139.817413, -0.118092, -73.935242, 55.296249, 114.177216, 72.877426, 100.523184, 2.349014, 28.979530, 127.024612]}

places = pd.DataFrame(data=data)
display(places)

Listed above are some popular locations with their listed latitude and longitude values. Once again, you can find the timezones for these locations using an online [timezone converter](https://www.timeanddate.com/worldclock/converter.html).

Great, we now have a reliable way to get a timezone from inputted longitude and latitude values. We also need to convert our times from the _12-hour_ format to the _24-hour_ format alongside using our local timezone which we want to switch our `sunrise` and `sunset` times into.

In [ ]:
def time_convert(time, timezone_swap:int):
    # Note, here you have to change the timezone information and adjust with a + or - accordingly
    t = datetime.strptime(time, '%I:%M:%S %p') + timedelta(hours=timezone_swap)
    # Format the datetime object into a 24-hour time string
    format = '%H:%M:%S'
    t = t.strftime(format)
    #  Convert back to datetime object
    t = datetime.strptime(t, '%H:%M:%S').time()
    return(t)

print(time_convert(sunrise, offset_hours))

By converting our data into a consistent format we make plotting later on much easier and it makes comparing different time values consistent as **24** will always be the maximum while **0** is always the minimum.

## Explore

Now that our data has been _transformed_, we can start finding ways to explore going beyond just finding `sunset` and `sunrise` times for a particular date. What if we wanted to look at the differences between `sunrise` and `sunset` in a scope of a couple of days, a month, or even a year? Let's try finding a way to set a particular range of dates in order to compare these values! 

(Note): When inputting ranges of dates that are over a year, the notebook will take a long time to load.

In [ ]:
total_info = {"date": []}

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

start_date = date(2023, 4, 19)
end_date = date(2023, 5, 19)

for single_date in daterange(start_date, end_date):
    total_info["date"].append(single_date.strftime("%Y-%m-%d"))
data = pd.DataFrame(total_info)

display(data)

Looking above, we see that each date is formatted as (YYYY-MM-DD), which is ideal for plotting as each date is in _sequential_ order. 

In the cell below, alter the corresponding `longitude`, `latitude` values alongside the particular number of hours needed to offset from the **UTC timezone**.

In [ ]:
# Alter these longitude and latitude values below

longitude = -114.06666 # change this longitude value
latitude = 51.049999   # change this latitude value

# Amount of hours to offset changed here - currently set to MST (Calgary's timezone)
offset_hours = -6

Putting it all together, we can now find `sunrise` and `sunset` times from a certain period of dates obtaining times in a _24-hour_ format.

In [ ]:
# params format: latitude, longitude, date (YYYY-MM-DD)
sunset_info = {"sunset": []}
sunrise_info = {"sunrise": []}
day_length = {"day_length": []}

def sunrisesunset(lat, lng, date):
    params = {"lat":lat, "lng":lng, "date":date}
    data = requests.get("https://api.sunrise-sunset.org/json", params=params)
    data = json.loads(data.text)
    data = data["results"]
    return data["sunrise"], data["sunset"], data["day_length"]

for i in range(len(data)):
    temp, temp1, temp2 = sunrisesunset(latitude, longitude, total_info['date'][i])
    sunrise_info['sunrise'].append(time_convert(temp, offset_hours))
    sunset_info['sunset'].append(time_convert(temp1, offset_hours))
    day_length['day_length'].append(temp2)

sunset_info = pd.DataFrame(sunset_info)
sunrise_info = pd.DataFrame(sunrise_info)
day_length = pd.DataFrame(day_length)
print("Sunset, Sunrise, and Day Lengths obtained successfully!")

Let's display everything in a **dateframe**, appending the `date`, `sunrise`, `sunset`, and `day_length` dataframes.

In [ ]:
temp_total_info = data.join(sunrise_info['sunrise'])
temp_total_info2 = temp_total_info.join(sunset_info["sunset"])
df = temp_total_info2.join(day_length['day_length'])
display(df)

From the information we gathered, we can now see _scatterplots_ for `sunrise`, `sunset`, and `day_length` values for our chosen timezone and day lengths.

In [ ]:
all_figs = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

sunset_fig = go.Scatter(x=df['date'], y=df['sunrise'], mode='lines+markers', name='Sunrise')
sunrise_fig = go.Scatter(x=df['date'], y=df['sunset'], mode='lines+markers', name='Sunset')
total_length = go.Scatter(x=df['date'], y=df['day_length'], mode='lines+markers', name='Day Length')

all_figs.add_trace(sunset_fig, row=1, col=1)
all_figs.add_trace(sunrise_fig, row=2, col=1)
all_figs.add_trace(total_length, row=3, col=1)

all_figs.update_layout(height=600,title_text="Sunrise, Sunset, and Day Length")

# Reverse y-axis if the initial value is greater than the next value
if df['sunrise'].iloc[0] > df['sunrise'].iloc[1]:
    all_figs.update_yaxes(autorange="reversed", row=1, col=1)

if df['sunset'].iloc[0] > df['sunset'].iloc[1]:
    all_figs.update_yaxes(autorange="reversed", row=2, col=1)

if df['day_length'].iloc[0] > df['day_length'].iloc[1]:
    all_figs.update_yaxes(autorange="reversed", row=3, col=1)

all_figs.update_layout(yaxis2=dict(title=dict(text='Time (24-hour format)')), xaxis3=dict(title=dict(text='Date'), tickmode='linear')).show()

## Interpret

In the scatter plot you can see the dates on the x-axis and the corresponding `sunrise`, `sunset`, and `day_lengths` on the y-axis. 

The color of the dots helps to differentiate the different plots with `sunrise` being red, `sunset` being blue, and `day_length` being green.

We notice that when the range of dates is set to approximately a month, `sunrise` and `sunset` times follow a _linear_ relationship, where the dates consistently either go up or down a particular value making the plot look like a straight line. Why do you think these values follow a certain _trend_?

## Reflect on what you see

After making your visualization the next step is to use the data and your visualization to answer the question. Look at and interact with the visualization above. When you hover your mouse over the plots, you’ll notice more information appears. You can also use the legend to make plots appear and disappear.

#### Think about the following questions.

* What do you notice about these graphs?
* What do you wonder about the data?
* What kind of inferences can you make based on this data?
* Is there another way to visualize this data that would change your inerpretation of the information? 


#### Use the fill-in-the-blank prompts to summarize your thoughts.
* "I used to think _______"
* "Now I think _______"
* "I wish I knew more about _______"
* "These data visualizations remind me of _______"
* "I really like _______"

## Communicate

If you have not yet done this, use the plot to answer our question on which timezones have the largest differences between sunrise and sunset times. Vice-versa, which areas have the smallest differences?

How can you communicate that information? What kind of product could you create to share that information with your school community and wider community?

Consider tagging Callysto on [Twitter](https://twitter.com/callysto_canada), [YouTube](https://www.youtube.com/Callysto), [TikTok](https://www.tiktok.com/@callysto_canada), [Facebook](https://www.facebook.com/callystocanada/), or [Linkedin](https://www.linkedin.com/company/callysto-canada/) if you decide to share your reflections or projects on social media.

## Further Resources

Other sources for sunrise and sunset times, such as the sunset and sunrise times of Calgary, can be found [here](https://www.timeanddate.com/sun/canada/calgary)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)